In [3]:
import pandas as pd



data = pd.read_csv(r'Data\soilpH_rgb.csv')
data

,R,G,B,pH
0,133,98,30,7.05
1,172,139,106,6.80
2,176,152,114,6.63
3,158,132,51,6.64
4,197,164,123,8.35
5,190,147,99,7.35
6,191,162,145,7.35
7,167,151,153,7.49
8,152,121,68,7.38
9,148,118,48,7.40


In [4]:
data['R/G'] = data['R']/data['G']
data['G/B'] = data['G']/data['B']
data['R/B'] = data['R']/data['B']


In [5]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with columns 'R', 'G', and 'B'
# Example DataFrame:
# df = pd.DataFrame({'R': [133, 172, 176, 158, 197],
#                    'G': [98, 139, 152, 132, 164],
#                    'B': [30, 106, 114, 51, 123]})

def rgb_to_hue(rgb):
    rgb = rgb / 255.0
    max_rgb = rgb.max(axis=1)
    min_rgb = rgb.min(axis=1)
    delta = max_rgb - min_rgb
    hue = np.zeros_like(max_rgb)
    non_zero_delta = delta != 0
    hue[non_zero_delta] = np.select(
        [max_rgb == rgb[:, 0], max_rgb == rgb[:, 1], max_rgb == rgb[:, 2]],
        [
            (rgb[:, 1] - rgb[:, 2]) / delta,
            2 + (rgb[:, 2] - rgb[:, 0]) / delta,
            4 + (rgb[:, 0] - rgb[:, 1]) / delta
        ]
    )
    hue = (hue * 60) % 360
    return hue

# Apply the function to create a new 'hue' column
data['hue'] = rgb_to_hue(data[['R', 'G', 'B']].values)

# Display the DataFrame with the new 'hue' column
data


,R,G,B,pH,R/G,G/B,R/B,hue
0,133,98,30,7.05,1.357143,3.266667,4.433333,39.611650
1,172,139,106,6.80,1.237410,1.311321,1.622642,30.000000
2,176,152,114,6.63,1.157895,1.333333,1.543860,36.774194
3,158,132,51,6.64,1.196970,2.588235,3.098039,45.420561
4,197,164,123,8.35,1.201220,1.333333,1.601626,33.243243
5,190,147,99,7.35,1.292517,1.484848,1.919192,31.648352
6,191,162,145,7.35,1.179012,1.117241,1.317241,22.173913
7,167,151,153,7.49,1.105960,0.986928,1.091503,352.500000
8,152,121,68,7.38,1.256198,1.779412,2.235294,37.857143
9,148,118,48,7.40,1.254237,2.458333,3.083333,42.000000


In [6]:
import pandas as pd
import colorsys

# Assuming df is your DataFrame with columns 'R', 'G', 'B'
# Add new columns for Saturation, Brightness, Opacity, and Temperature
data['Saturation'] = data.apply(lambda row: colorsys.rgb_to_hsv(row['R']/255, row['G']/255, row['B']/255)[1], axis=1)
data['Brightness'] = data.apply(lambda row: colorsys.rgb_to_hsv(row['R']/255, row['G']/255, row['B']/255)[2], axis=1)

#data['Temperature'] = data.apply(lambda row: 1 if row['R'] > row['B'] else 0, axis=1)

# Display the DataFrame with the new columns
data


,R,G,B,pH,R/G,G/B,R/B,hue,Saturation,Brightness
0,133,98,30,7.05,1.357143,3.266667,4.433333,39.611650,0.774436,0.521569
1,172,139,106,6.80,1.237410,1.311321,1.622642,30.000000,0.383721,0.674510
2,176,152,114,6.63,1.157895,1.333333,1.543860,36.774194,0.352273,0.690196
3,158,132,51,6.64,1.196970,2.588235,3.098039,45.420561,0.677215,0.619608
4,197,164,123,8.35,1.201220,1.333333,1.601626,33.243243,0.375635,0.772549
5,190,147,99,7.35,1.292517,1.484848,1.919192,31.648352,0.478947,0.745098
6,191,162,145,7.35,1.179012,1.117241,1.317241,22.173913,0.240838,0.749020
7,167,151,153,7.49,1.105960,0.986928,1.091503,352.500000,0.095808,0.654902
8,152,121,68,7.38,1.256198,1.779412,2.235294,37.857143,0.552632,0.596078
9,148,118,48,7.40,1.254237,2.458333,3.083333,42.000000,0.675676,0.580392


In [ ]:
data.sort_values(by = 'gray')

In [7]:
X = data.drop('pH',axis=1)
y = data['pH']

In [8]:
X

,R,G,B,R/G,G/B,R/B,hue,Saturation,Brightness
0,133,98,30,1.357143,3.266667,4.433333,39.611650,0.774436,0.521569
1,172,139,106,1.237410,1.311321,1.622642,30.000000,0.383721,0.674510
2,176,152,114,1.157895,1.333333,1.543860,36.774194,0.352273,0.690196
3,158,132,51,1.196970,2.588235,3.098039,45.420561,0.677215,0.619608
4,197,164,123,1.201220,1.333333,1.601626,33.243243,0.375635,0.772549
5,190,147,99,1.292517,1.484848,1.919192,31.648352,0.478947,0.745098
6,191,162,145,1.179012,1.117241,1.317241,22.173913,0.240838,0.749020
7,167,151,153,1.105960,0.986928,1.091503,352.500000,0.095808,0.654902
8,152,121,68,1.256198,1.779412,2.235294,37.857143,0.552632,0.596078
9,148,118,48,1.254237,2.458333,3.083333,42.000000,0.675676,0.580392


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# Assuming df is your DataFrame with the given data
# X contains the RGB values, and y contains the pH values
X = data.drop('pH',axis=1)
y = data['pH']


# Create a CatBoost regressor
model = CatBoostRegressor(silent=True)

# Define a custom scoring function for RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Use cross_val_score to perform cross-validation and calculate RMSE
cv_scores = cross_val_score(model, X, y, cv=5, scoring=make_scorer(rmse))

# Print the cross-validated RMSE scores
print("Cross-Validated RMSE Scores:", cv_scores)

# Print the average RMSE across folds
print("Average RMSE:", np.mean(cv_scores))


In [10]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# Assuming df is your DataFrame with the given data
# X contains the RGB values, and y contains the pH values
#X = data.drop('pH', axis=1)
#y = data['pH']

# Create an XGBoost regressor
model = XGBRegressor(objective='reg:squarederror')  # Use 'reg:squarederror' for regression problems

# Define a custom scoring function for RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Use cross_val_score to perform cross-validation and calculate RMSE
cv_scores = cross_val_score(model, X, y, cv=5, scoring=make_scorer(rmse))

# Print the cross-validated RMSE scores
print("Cross-Validated RMSE Scores:", cv_scores)

# Print the average RMSE across folds
print("Average RMSE:", np.mean(cv_scores))


Cross-Validated RMSE Scores: [0.72568694 0.59847474 0.48464535 0.67740221 1.03809746]
Average RMSE: 0.7048613416602851


In [ ]:
import pickle
model.fit(X, y)

# Save the trained model to a pickle file
with open('soil_colour_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

print("Model trained and saved as 'soil_colour_model.pkl'.")

In [ ]:
import matplotlib.pyplot as plt

def display_color(rgb_values):
    # Ensure that the RGB values are in the range [0, 255]
    rgb_values = [max(0, min(255, value)) for value in rgb_values]

    # Normalize the RGB values to the range [0, 1]
    normalized_values = [value / 255.0 for value in rgb_values]

    # Display the color using matplotlib
    plt.imshow([[normalized_values]])
    plt.axis('off')
    plt.show()

# Example usage
rgb_values = [132, 98, 30]  # Red color
display_color(rgb_values)







In [ ]:
def rgb_to_grayscale(rgb_values):
    # Ensure that the RGB values are in the range [0, 255]
    rgb_values = [max(0, min(255, value)) for value in rgb_values]

    # Convert RGB to grayscale using the specified formula
    grayscale_value = 0.299 * rgb_values[0] + 0.587 * rgb_values[1] + 0.114 * rgb_values[2]

    return int(grayscale_value)

# Example usage
rgb_values = [133, 98, 30]  # Red color
grayscale = rgb_to_grayscale(rgb_values)
print(f"The grayscale value is: {grayscale}")


In [ ]:
def map_values(x, x1, y1, x2, y2):
    # Ensure x is within the range [x1, x2]
   

    # Linear interpolation formula
    y = y1 + ((x - x1) * (y2 - y1)) / (x2 - x1)

    return y

# Example usage
x_value = 0.17

mapped_value = map_values(x_value, 0.12, 25, 0.65, 250)
print(f"The mapped value for {x_value} is: {mapped_value}")


In [ ]:


def get_SoilProfile(R,G,B):
    rgb = {
        'R' : R,
        'G' : G,
        'B' : B
    }
    rgb_data = pd.DataFrame.from_dict({0: rgb}, orient='index')
    rgb_data['R/G'] = rgb_data['R']/rgb_data['G']
    rgb_data['G/B'] = rgb_data['G']/rgb_data['B']
    rgb_data['R/B'] = rgb_data['R']/rgb_data['B']

    def rgb_to_hue(rgb):
        rgb = rgb / 255.0
        max_rgb = rgb.max(axis=1)
        min_rgb = rgb.min(axis=1)
        delta = max_rgb - min_rgb
        hue = np.zeros_like(max_rgb)
        non_zero_delta = delta != 0
        hue[non_zero_delta] = np.select(
            [max_rgb == rgb[:, 0], max_rgb == rgb[:, 1], max_rgb == rgb[:, 2]],
            [
                (rgb[:, 1] - rgb[:, 2]) / delta,
                2 + (rgb[:, 2] - rgb[:, 0]) / delta,
                4 + (rgb[:, 0] - rgb[:, 1]) / delta
            ]
        )
        hue = (hue * 60) % 360
        return hue



    # Apply the function to create a new 'hue' column
    rgb_data['hue'] = rgb_to_hue(rgb_data[['R', 'G', 'B']].values)

    # Assuming df is your DataFrame with columns 'R', 'G', 'B'
    # Add new columns for Saturation, Brightness, Opacity, and Temperature
    rgb_data['Saturation'] = rgb_data.apply(lambda row: colorsys.rgb_to_hsv(row['R']/255, row['G']/255, row['B']/255)[1], axis=1)
    rgb_data['Brightness'] = rgb_data.apply(lambda row: colorsys.rgb_to_hsv(row['R']/255, row['G']/255, row['B']/255)[2], axis=1)


    soil_pH = model.predict(rgb_data)[0]

    gray_scale = 1- ((R*0.299 + G*0.582 + B*0.114)/255)
    
    # Ensure x is within the range [x1, x2

    # Linear interpolation formula
    water_availabiliy = 25 + ((gray_scale - 0.12) * (250 - 25)) / (0.65 - 0.12)
    return soil_pH,water_availabiliy

In [ ]:
get_SoilProfile(140,20,111)